In [ ]:
import os  
import base64
from openai import AzureOpenAI  
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI

endpoint = os.getenv("", "")  
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")  
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "")  

# Initialize Azure OpenAI Service client with key-based authentication    
client = AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version = "2024-12-01-preview"
)

    
# Generate the completion  
def chat_completion_4o(USER_MSG):
    completion = client.chat.completions.create(  
        model=deployment,
        messages=[
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "You are going to identify the corresponding sense tag of an ambiguous word."
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": USER_MSG
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": ""
            }
        ]
    }
] ,
        max_tokens=500,  
        temperature=0.0,
        top_p=0.95,  
        frequency_penalty=0,  
        presence_penalty=0,
        stop=None,  
        stream=False
        )
    response_content = completion.choices[0].message.content
    return response_content

In [ ]:
import os
from together import Together

client = Together() # auth defaults to os.environ.get("TOGETHER_API_KEY")

def chat_completion_deepseek(user_msg: str) -> str:   

  response = client.chat.completions.create(
      model="deepseek-ai/DeepSeek-V3",
      messages=[
        {
          "role": "user",
          "content": user_msg
        }
      ]
  )
  return(response.choices[0].message.content)


In [ ]:
# identify the meaning of the word using GLossGPT prompt template

from openai import OpenAI

client = OpenAI()

def get_model_output(Question):
  response = client.chat.completions.create(
    model='virtuoso-large',
    messages=[{'role': 'user', 'content': Question}],
    temperature=0.4,
  )

  return(response.choices[0].message.content)

In [ ]:
import google.generativeai as genai

# Configure the API key
genai.configure(api_key="") # Or it will pick up from GEMINI_API_KEY env var

# Initialize the model
model = genai.GenerativeModel('gemini-2.5-flash')

def get_gemini_response(question):

    response = model.generate_content(question)   
    return(response.text)


In [ ]:
#BASELINE For the Test Cases

In [ ]:
import csv
def main():
    file_path = 'Updated_test.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"

            prompt=f'''Your task is to rate the plausibility of a word's meaning on a scale of 1-5 based on a short story.

**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

**Instructions:**
1.  **Analyze the Context:** Read the complete story and identify all clues that might support or contradict the 'Proposed Meaning'.
2.  **List Evidence For:** State the parts of the story that make the 'Proposed Meaning' plausible.
3.  **List Evidence Against:** State any parts of the story that make the 'Proposed Meaning' implausible.
4.  **Synthesize and Score:** Based on the evidence, provide a final plausibility score using the rubric below.

**Scoring Rubric:**
- **5:** Perfectly plausible. The meaning is strongly and directly supported by multiple clues in the context.
- **4:** Very plausible. The meaning fits well and is consistent with the context.
- **3:** Moderately plausible. The meaning is possible but not strongly implied; the context is ambiguous.
- **2:** Barely plausible. The meaning largely conflicts with the context but isn't entirely impossible.
- **1:** Implausible. The meaning is directly contradicted by the context.

Do all the reasoning internally. Provide only the final integer score '''                       
        
                    
            # Generate response using the same format as training
            full_text = get_gemini_response(prompt)
            clean_text = full_text.replace('\n', ' ')
            print(f"Line {line_num}: {clean_text}")
            line_num += 1


if __name__ == "__main__":
    main()

In [ ]:
import csv
def main():
    file_path = 'Updated_test.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"

            prompt=f'''Your task is to rate the plausibility of a word's meaning on a scale of 1-5 based on a short story.

**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

**Instructions:**
1.  **Analyze the Context:** Read the complete story and identify all clues that might support or contradict the 'Proposed Meaning'.
2.  **List Evidence For:** State the parts of the story that make the 'Proposed Meaning' plausible.
3.  **List Evidence Against:** State any parts of the story that make the 'Proposed Meaning' implausible.
4.  **Synthesize and Score:** Based on the evidence, provide a final plausibility score using the rubric below.

**Scoring Rubric:**
- **5:** Perfectly plausible. The meaning is strongly and directly supported by multiple clues in the context.
- **4:** Very plausible. The meaning fits well and is consistent with the context.
- **3:** Moderately plausible. The meaning is possible but not strongly implied; the context is ambiguous.
- **2:** Barely plausible. The meaning largely conflicts with the context but isn't entirely impossible.
- **1:** Implausible. The meaning is directly contradicted by the context.

Do all the reasoning internally. Provide only the final integer score '''                       
        
                    
            # Generate response using the same format as training
            full_text = chat_completion_4o(prompt)
            clean_text = full_text.replace('\n', ' ')
            print(f"Line {line_num}: {clean_text}")
            line_num += 1


if __name__ == "__main__":
    main()

In [ ]:
import csv
def main():
    file_path = 'Updated_test.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"

            prompt=f'''Your task is to rate the plausibility of a word's meaning on a scale of 1-5 based on a short story.

**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

**Instructions:**
1.  **Analyze the Context:** Read the complete story and identify all clues that might support or contradict the 'Proposed Meaning'.
2.  **List Evidence For:** State the parts of the story that make the 'Proposed Meaning' plausible.
3.  **List Evidence Against:** State any parts of the story that make the 'Proposed Meaning' implausible.
4.  **Synthesize and Score:** Based on the evidence, provide a final plausibility score using the rubric below.

**Scoring Rubric:**
- **5:** Perfectly plausible. The meaning is strongly and directly supported by multiple clues in the context.
- **4:** Very plausible. The meaning fits well and is consistent with the context.
- **3:** Moderately plausible. The meaning is possible but not strongly implied; the context is ambiguous.
- **2:** Barely plausible. The meaning largely conflicts with the context but isn't entirely impossible.
- **1:** Implausible. The meaning is directly contradicted by the context.

Do all the reasoning internally. Provide only the final integer score '''                       
        
                    
            # Generate response using the same format as training
            full_text = chat_completion_deepseek(prompt)
            clean_text = full_text.replace('\n', ' ')
            print(f"Line {line_num}: {clean_text}")
            line_num += 1


if __name__ == "__main__":
    main()

In [ ]:
import csv
def main():
    file_path = 'Updated_dev.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"

            prompt=f'''our task is to rate the plausibility of a word's meaning on a scale of 1-5 based on a short story.

**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

**Instructions:**
1.  **Analyze the Context:** Read the complete story and identify all clues that might support or contradict the 'Proposed Meaning'.
2.  **List Evidence For:** State the parts of the story that make the 'Proposed Meaning' plausible.
3.  **List Evidence Against:** State any parts of the story that make the 'Proposed Meaning' implausible.
4.  **Synthesize and Score:** Based on the evidence, provide a final plausibility score using the rubric below.

**Scoring Rubric:**
- **5:** Perfectly plausible. The meaning is strongly and directly supported by multiple clues in the context.
- **4:** Very plausible. The meaning fits well and is consistent with the context.
- **3:** Moderately plausible. The meaning is possible but not strongly implied; the context is ambiguous.
- **2:** Barely plausible. The meaning largely conflicts with the context but isn't entirely impossible.
- **1:** Implausible. The meaning is directly contradicted by the context.

Provide only the final integer score after your reasoning.'''                       
        
                    
            # Generate response using the same format as training
            full_text = get_model_output(prompt)
            print(f"Line {line_num}: {full_text.replace('\n', ' ')}")
            line_num += 1


if __name__ == "__main__":
    main()

In [ ]:
import json

def save_predictions_to_jsonl(numbers, file_path):
    """
    Saves a list of numbers to a JSONL file in the format:
    {"id": "<index>", "prediction": <number>}
    
    Args:
        numbers (list): List of numbers (predictions).
        file_path (str): Path to output .jsonl file.
    """
    with open(file_path, 'w', encoding='utf-8') as f:
        for idx, num in enumerate(numbers):
            record = {
                "id": str(idx),
                "prediction": num
            }
            f.write(json.dumps(record) + "\n")

In [ ]:
predicts=[]
with open("test_results/gpt40.txt", "r", encoding="utf-8") as f:
    for i in f:
        elem = i.strip().split(" ")
        if elem[0] == "Line":
            predicts.append(int(elem[2]))
print(predicts)
print(len(predicts))
# save_predictions_to_jsonl(predicts, "predictions.jsonl")

In [ ]:
import pandas as pd

gold_final_score = []
with open('dev.csv', 'r', encoding='utf-8') as f:
    df = pd.read_csv(f)
    for index, row in df.iterrows():
        average = row['average']
        gold_final_score.append(average)       
        

print(gold_final_score)
print(len(gold_final_score))

In [ ]:
difference = 0
for gold, predict in zip(gold_final_score, predicts):
    diff = abs(gold - predict)
    difference += diff
    
print(difference/len(gold_final_score))

from scipy.stats import spearmanr

# gold_final_score and Gemma_final_score are lists of equal length
spearman_corr, _ = spearmanr(gold_final_score, predicts)
print(f"Spearman Correlation: {spearman_corr:.4f}")


import numpy as np

gold_array = np.array(gold_final_score)
pred_array = np.array(predicts)

std_dev = np.std(gold_array)  # global standard deviation across gold scores
tolerance = std_dev  # "within 1 std deviation"

within_std = np.abs(pred_array - gold_array) <= tolerance
accuracy_within_std = np.mean(within_std)

print(f"Standard Deviation: {std_dev:.4f}")
print(f"Accuracy within 1 Std Dev: {accuracy_within_std:.4f}")

In [ ]:
import csv
def main():
    file_path = 'Updated_train_with_scores.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]
            average = round(float(average))
            human_label =row["human_label"]

            if ending.strip() == "":
                ending = "[NO ENDING]"

            if human_label == "HumanUnlikely":
                prompt=f'''
**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Homonym Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"
**Plausibility Score:** {average}'''
        
                    
            # Generate response using the same format as training
                prompt_updated = prompt.replace('\n', ' ')
                print(f"{prompt_updated}")
            else:
                continue
            


if __name__ == "__main__":
    main()

# Gemini Zero Shot

In [ ]:
import csv
import time
def main():
    file_path = 'Updated_dev.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"

            prompt=f'''our task is to rate the plausibility of a word's meaning on a scale of 1-5 based on a short story.

**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

**Instructions:**
1.  **Analyze the Context:** Read the complete story and identify all clues that might support or contradict the 'Proposed Meaning'.
2.  **List Evidence For:** State the parts of the story that make the 'Proposed Meaning' plausible.
3.  **List Evidence Against:** State any parts of the story that make the 'Proposed Meaning' implausible.
4.  **Synthesize and Score:** Based on the evidence, provide a final plausibility score using the rubric below.

**Scoring Rubric:**
- **5:** Perfectly plausible. The meaning is strongly and directly supported by multiple clues in the context.
- **4:** Very plausible. The meaning fits well and is consistent with the context.
- **3:** Moderately plausible. The meaning is possible but not strongly implied; the context is ambiguous.
- **2:** Barely plausible. The meaning largely conflicts with the context but isn't entirely impossible.
- **1:** Implausible. The meaning is directly contradicted by the context.

Provide only the final integer score after your reasoning.'''                       
        
                    
            # Generate response using the same format as training
            full_text = get_model_output_gemini(prompt)
            output_updated = full_text.replace('\n', ' ')             
            print(f"Line {line_num}: {output_updated}")
            line_num += 1
            time.sleep(20)


if __name__ == "__main__":
    main()

In [ ]:
import re

def get_number_at_end(sentence: str):
    """
    Returns the number found at the end of a given sentence.
    Handles cases like **5**, (5), or 5.
    Returns None if no number is found.
    """
    match = re.search(r'[\*\(\[\{]*(-?\d+(?:\.\d+)?)[\*\)\]\}]*\s*$', sentence)
    if match:
        num_str = match.group(1)
        # Return int if it’s whole, float otherwise
        return int(num_str) if num_str.isdigit() or (num_str.startswith('-') and num_str[1:].isdigit()) else float(num_str)
    return None

In [ ]:
predicts=[]
with open("zeroshot_Gemini.txt", "r", encoding="utf-8") as f:
    for i in f:
        no = get_number_at_end(i.strip())
        predicts.append(no)
        #(no)
print(predicts)
print(len(predicts))
#save_predictions_to_jsonl(predicts, "predictions.jsonl")

In [ ]:
difference = 0
for gold, predict in zip(gold_final_score, predicts):
    diff = abs(gold - predict)
    difference += diff
    
print(difference/len(gold_final_score))

from scipy.stats import spearmanr

# gold_final_score and Gemma_final_score are lists of equal length
spearman_corr, _ = spearmanr(gold_final_score, predicts)
print(f"Spearman Correlation: {spearman_corr:.4f}")


import numpy as np

gold_array = np.array(gold_final_score)
pred_array = np.array(predicts)

std_dev = np.std(gold_array)  # global standard deviation across gold scores
tolerance = std_dev  # "within 1 std deviation"

within_std = np.abs(pred_array - gold_array) <= tolerance
accuracy_within_std = np.mean(within_std)

print(f"Standard Deviation: {std_dev:.4f}")
print(f"Accuracy within 1 Std Dev: {accuracy_within_std:.4f}")

In [ ]:
# create a 3 vector strores for each case
# extract one example based on the input
# create a few shot example to simulate the few shot output.

# Single annotator agreement with Virtuso-large

In [ ]:
from huggingface_hub import login
login("")

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import os

In [ ]:
# Files and labels
file_paths = {
    "ambiguous": "human_ambigous.txt",
    "easy": "human_easy.txt",
    "unlikely": "human_unlikely.txt"
}

In [ ]:
def load_and_split(file_path):
    """Load and split a text file using \n as separator."""
    loader = TextLoader(file_path, encoding='utf-8')
    documents = loader.load()
    
    splitter = RecursiveCharacterTextSplitter(
        separators=["\n"],
        chunk_size=500,
        chunk_overlap=50
    )
    split_docs = splitter.split_documents(documents)
    return split_docs

In [ ]:
!hf auth whoami


In [ ]:
def create_vector_store(docs, store_name):
    """Create and save a FAISS vector store using BGE-small embeddings."""
    embedding_model = HuggingFaceEmbeddings(
        model_name="BAAI/bge-small-en-v1.5",
        model_kwargs={'device': 'cpu'},  # change to 'cuda' if you have GPU
        encode_kwargs={'normalize_embeddings': True}
    )
    
    vector_store = FAISS.from_documents(docs, embedding_model)
    vector_store.save_local(f"vectorstore_{store_name}")
    return vector_store



In [ ]:
vector_stores = {}
for label, path in file_paths.items():
    docs = load_and_split(path)
    vector_stores[label] = create_vector_store(docs, label)

In [ ]:
def retrieve_from_all_stores(query, k=1):
    """Retrieve top-k results from each vector store."""
    results = {}
    for label, store in vector_stores.items():
        docs = store.similarity_search(query, k=k)
        results[label] = docs
    return results

In [ ]:
def load_vector_store(store_name):
    """Load an existing FAISS vector store from disk."""
    embedding_model = HuggingFaceEmbeddings(
        model_name="BAAI/bge-small-en-v1.5",
        model_kwargs={'device': 'cpu'},  # change to 'cuda' if you have GPU
        encode_kwargs={'normalize_embeddings': True}
    )
    
    return FAISS.load_local(
        f"vectorstore_{store_name}",
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  # required for FAISS in some LangChain versions
    )

# -------------------------------
# Step 3: Load all your stores
# -------------------------------
vector_stores = {
    "ambiguous": load_vector_store("ambiguous"),
    "easy": load_vector_store("easy"),
    "unlikely": load_vector_store("unlikely")
}

In [ ]:
def rag_pipeline(query):
    """
    Retrieve one document (k=1) from each vector store
    and return them as strings.
    """
    retrieved = retrieve_from_all_stores(query, k=1)

    # Create a dict of store_name -> retrieved text
    retrieved_texts = {}
    for label, docs in retrieved.items():
        if docs:
            retrieved_texts[label] = docs[0].page_content.strip()
        else:
            retrieved_texts[label] = ""

    # Optionally, combine all into a single string
    combined_text = "\n\n".join(
        [f"### {label.upper()} CONTEXT ###\n{content}"
         for label, content in retrieved_texts.items()]
    )

    return combined_text, retrieved_texts

In [ ]:
def rag_pipeline(query, k=2):
    """
    Retrieve top-k documents from each vector store
    and return them as combined and structured strings.
    """
    retrieved = retrieve_from_all_stores(query, k=k)

    retrieved_texts = {}
    for label, docs in retrieved.items():
        if docs:
            # Join all retrieved chunks for that store
            combined_chunks = "\n---\n".join([d.page_content.strip() for d in docs])
            retrieved_texts[label] = combined_chunks
        else:
            retrieved_texts[label] = ""

    # Combine everything into one formatted string
    combined_text = "\n\n".join(
        [f"### {label.upper()} CONTEXT ###\n{content}"
         for label, content in retrieved_texts.items() if content]
    )

    return combined_text, retrieved_texts

In [ ]:
def rag_pipeline(query, k=3):
    """
    Retrieve top-k documents from each vector store
    and return them as combined and structured strings.
    """
    retrieved = retrieve_from_all_stores(query, k=k)

    retrieved_texts = {}
    for label, docs in retrieved.items():
        if docs:
            # Join all retrieved chunks for that store
            combined_chunks = "\n---\n".join([d.page_content.strip() for d in docs])
            retrieved_texts[label] = combined_chunks
        else:
            retrieved_texts[label] = ""

    # Combine everything into one formatted string
    combined_text = "\n\n".join(
        [f"### {label.upper()} CONTEXT ###\n{content}"
         for label, content in retrieved_texts.items() if content]
    )

    return combined_text, retrieved_texts

In [ ]:
query = "a pair of parallel rails providing a runway for wheels,The detectives arrived at the abandoned train station. They were looking for signs of the missing artifact. A faint trail caught their attention.,They followed the <WSD>track</WSD>.,They began to run along the abandoned railway line, hopping from wooden sleeper to sleeper to avoid twisting an ankle.How do humans react to ambiguous situations?"
combined_text, retrieved_texts = rag_pipeline(query, k=1)

print("🔹 Combined Text:\n")
print(combined_text)


In [ ]:
import csv
def main():
    file_path = 'Updated_test.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"


            pre_context = f''' **Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Homonym Word:** "{homonym}'''
            
            combined_text, retrieved_texts = rag_pipeline(pre_context)

            prompt=f'''our task is to rate the plausibility of a word's meaning on a scale of 1-5 based on a short story.

            You will be given:
- A short story (including *Precontext*, *Sentence*, and *Ending*)
- A *target word* (homonym)
- A *proposed meaning* for that word

Below are examples that illustrate the scoring rubric for different levels of plausibility, based on similar stories: {combined_text}

Now evaluate the following story and proposed meaning:
**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

**Instructions:**
1.  **Analyze the Context:** Read the complete story and identify all clues that might support or contradict the 'Proposed Meaning'.
2.  **List Evidence For:** State the parts of the story that make the 'Proposed Meaning' plausible.
3.  **List Evidence Against:** State any parts of the story that make the 'Proposed Meaning' implausible.
4.  **Synthesize and Score:** Based on the evidence, provide a final plausibility score using the rubric below.
5.  Return only the final integer score.

**Scoring Rubric:**
- **5:** Perfectly plausible. The meaning is strongly and directly supported by multiple clues in the context.
- **4:** Very plausible. The meaning fits well and is consistent with the context.
- **3:** Moderately plausible. The meaning is possible but not strongly implied; the context is ambiguous.
- **2:** Barely plausible. The meaning largely conflicts with the context but isn't entirely impossible.
- **1:** Implausible. The meaning is directly contradicted by the context.


Perform a proper reasoning and only provide the final integer. Any explanation and discussions are not expected. Expected output is only a single integer score from 1 to 5.'''                       
        
                    
            # Generate response using the same format as training
            full_text = get_gemini_response(prompt)
            output_updated = full_text.replace('\n', ' ')             
            print(f"Line {line_num}: {output_updated}")
            line_num += 1


if __name__ == "__main__":
    main()

In [ ]:
import csv
def main():
    file_path = 'Updated_test.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"


            pre_context = f''' **Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Homonym Word:** "{homonym}'''
            
            combined_text, retrieved_texts = rag_pipeline(pre_context)

            prompt=f'''our task is to rate the plausibility of a word's meaning on a scale of 1-5 based on a short story.

            You will be given:
- A short story (including *Precontext*, *Sentence*, and *Ending*)
- A *target word* (homonym)
- A *proposed meaning* for that word

Below are examples that illustrate the scoring rubric for different levels of plausibility, based on similar stories: {combined_text}

Now evaluate the following story and proposed meaning:
**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

**Instructions:**
1.  **Analyze the Context:** Read the complete story and identify all clues that might support or contradict the 'Proposed Meaning'.
2.  **List Evidence For:** State the parts of the story that make the 'Proposed Meaning' plausible.
3.  **List Evidence Against:** State any parts of the story that make the 'Proposed Meaning' implausible.
4.  **Synthesize and Score:** Based on the evidence, provide a final plausibility score using the rubric below.

**Scoring Rubric:**
- **5:** Perfectly plausible. The meaning is strongly and directly supported by multiple clues in the context.
- **4:** Very plausible. The meaning fits well and is consistent with the context.
- **3:** Moderately plausible. The meaning is possible but not strongly implied; the context is ambiguous.
- **2:** Barely plausible. The meaning largely conflicts with the context but isn't entirely impossible.
- **1:** Implausible. The meaning is directly contradicted by the context.


Perform a proper reasoning and only provide the final integer. Any explanation and discussions are not expected..'''                       
        
                    
            # Generate response using the same format as training
            full_text = chat_completion_deepseek(prompt)
            output_updated = full_text.replace('\n', ' ')             
            print(f"Line {line_num}: {output_updated}")
            line_num += 1


if __name__ == "__main__":
    main()

In [ ]:
import csv
def main():
    file_path = 'Updated_test.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"


            pre_context = f''' **Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Homonym Word:** "{homonym}'''
            
            combined_text, retrieved_texts = rag_pipeline(pre_context)

            prompt=f'''our task is to rate the plausibility of a word's meaning on a scale of 1-5 based on a short story.

            You will be given:
- A short story (including *Precontext*, *Sentence*, and *Ending*)
- A *target word* (homonym)
- A *proposed meaning* for that word

Below are examples that illustrate the scoring rubric for different levels of plausibility, based on similar stories: {combined_text}

Now evaluate the following story and proposed meaning:
**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

**Instructions:**
1.  **Analyze the Context:** Read the complete story and identify all clues that might support or contradict the 'Proposed Meaning'.
2.  **List Evidence For:** State the parts of the story that make the 'Proposed Meaning' plausible.
3.  **List Evidence Against:** State any parts of the story that make the 'Proposed Meaning' implausible.
4.  **Synthesize and Score:** Based on the evidence, provide a final plausibility score using the rubric below.

**Scoring Rubric:**
- **5:** Perfectly plausible. The meaning is strongly and directly supported by multiple clues in the context.
- **4:** Very plausible. The meaning fits well and is consistent with the context.
- **3:** Moderately plausible. The meaning is possible but not strongly implied; the context is ambiguous.
- **2:** Barely plausible. The meaning largely conflicts with the context but isn't entirely impossible.
- **1:** Implausible. The meaning is directly contradicted by the context.


Perform a proper reasoning and only provide the final integer. Any explanation and discussions are not expected..'''                       
        
                    
            # Generate response using the same format as training
            full_text = chat_completion_4o(prompt)
            output_updated = full_text.replace('\n', ' ')             
            print(f"Line {line_num}: {output_updated}")
            line_num += 1


if __name__ == "__main__":
    main()

In [ ]:
import csv
def main():
    file_path = 'Updated_test.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"


            pre_context = f''' **Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Homonym Word:** "{homonym}'''
            
            combined_text, retrieved_texts = rag_pipeline(pre_context)

            prompt=f'''our task is to rate the plausibility of a word's meaning on a scale of 1-5 based on a short story.

            You will be given:
- A short story (including *Precontext*, *Sentence*, and *Ending*)
- A *target word* (homonym)
- A *proposed meaning* for that word

Below are examples that illustrate the scoring rubric for different levels of plausibility, based on similar stories: {combined_text}

Now evaluate the following story and proposed meaning:
**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

**Instructions:**
1.  **Analyze the Context:** Read the complete story and identify all clues that might support or contradict the 'Proposed Meaning'.
2.  **List Evidence For:** State the parts of the story that make the 'Proposed Meaning' plausible.
3.  **List Evidence Against:** State any parts of the story that make the 'Proposed Meaning' implausible.
4.  **Synthesize and Score:** Based on the evidence, provide a final plausibility score using the rubric below.

**Scoring Rubric:**
- **5:** Perfectly plausible. The meaning is strongly and directly supported by multiple clues in the context.
- **4:** Very plausible. The meaning fits well and is consistent with the context.
- **3:** Moderately plausible. The meaning is possible but not strongly implied; the context is ambiguous.
- **2:** Barely plausible. The meaning largely conflicts with the context but isn't entirely impossible.
- **1:** Implausible. The meaning is directly contradicted by the context.


Perform a proper reasoning and only provide the final integer. Any explanation and discussions are not expected..'''                       
        
                    
            # Generate response using the same format as training
            full_text = chat_completion_4o(prompt)
            output_updated = full_text.replace('\n', ' ')             
            print(f"Line {line_num}: {output_updated}")
            line_num += 1


if __name__ == "__main__":
    main()

In [ ]:
import csv
def main():
    file_path = 'Updated_test.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"


            pre_context = f''' **Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Homonym Word:** "{homonym}'''
            
            combined_text, retrieved_texts = rag_pipeline(pre_context)

            prompt=f'''our task is to rate the plausibility of a word's meaning on a scale of 1-5 based on a short story.

            You will be given:
- A short story (including *Precontext*, *Sentence*, and *Ending*)
- A *target word* (homonym)
- A *proposed meaning* for that word

Below are examples that illustrate the scoring rubric for different levels of plausibility, based on similar stories: {combined_text}

Now evaluate the following story and proposed meaning:
**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

**Instructions:**
1.  **Analyze the Context:** Read the complete story and identify all clues that might support or contradict the 'Proposed Meaning'.
2.  **List Evidence For:** State the parts of the story that make the 'Proposed Meaning' plausible.
3.  **List Evidence Against:** State any parts of the story that make the 'Proposed Meaning' implausible.
4.  **Synthesize and Score:** Based on the evidence, provide a final plausibility score using the rubric below.

**Scoring Rubric:**
- **5:** Perfectly plausible. The meaning is strongly and directly supported by multiple clues in the context.
- **4:** Very plausible. The meaning fits well and is consistent with the context.
- **3:** Moderately plausible. The meaning is possible but not strongly implied; the context is ambiguous.
- **2:** Barely plausible. The meaning largely conflicts with the context but isn't entirely impossible.
- **1:** Implausible. The meaning is directly contradicted by the context.


Perform a proper reasoning and only provide the final integer. Any explanation and discussions are not expected..'''                       
        
                    
            # Generate response using the same format as training
            full_text = get_model_output_gemini(prompt)
            output_updated = full_text.replace('\n', ' ')             
            print(f"Line {line_num}: {output_updated}")
            line_num += 1


if __name__ == "__main__":
    main()

In [ ]:
import csv
def main():
    file_path = 'Updated_test.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"


            pre_context = f''' **Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Homonym Word:** "{homonym}'''
            
            combined_text, retrieved_texts = rag_pipeline(pre_context)

            prompt=f'''our task is to rate the plausibility of a word's meaning on a scale of 1-5 based on a short story.

            You will be given:
- A short story (including *Precontext*, *Sentence*, and *Ending*)
- A *target word* (homonym)
- A *proposed meaning* for that word

Below are examples that illustrate the scoring rubric for different levels of plausibility, based on similar stories: {combined_text}

Now evaluate the following story and proposed meaning:
**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

**Instructions:**
1.  **Analyze the Context:** Read the complete story and identify all clues that might support or contradict the 'Proposed Meaning'.
2.  **List Evidence For:** State the parts of the story that make the 'Proposed Meaning' plausible.
3.  **List Evidence Against:** State any parts of the story that make the 'Proposed Meaning' implausible.
4.  **Synthesize and Score:** Based on the evidence, provide a final plausibility score using the rubric below.

**Scoring Rubric:**
- **5:** Perfectly plausible. The meaning is strongly and directly supported by multiple clues in the context.
- **4:** Very plausible. The meaning fits well and is consistent with the context.
- **3:** Moderately plausible. The meaning is possible but not strongly implied; the context is ambiguous.
- **2:** Barely plausible. The meaning largely conflicts with the context but isn't entirely impossible.
- **1:** Implausible. The meaning is directly contradicted by the context.


Perform a proper reasoning and only provide the final integer. Any explanation and discussions are not expected..'''                       
        
                    
            # Generate response using the same format as training
            full_text = chat_completion_deepseek(prompt)
            output_updated = full_text.replace('\n', ' ')             
            print(f"Line {line_num}: {output_updated}")
            line_num += 1


if __name__ == "__main__":
    main()

In [ ]:
import csv
def main():
    file_path = 'Updated_dev.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"


            pre_context = f''' **Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Homonym Word:** "{homonym}'''
            
            combined_text, retrieved_texts = rag_pipeline(pre_context)

            prompt=f'''our task is to rate the plausibility of a word's meaning on a scale of 1-5 based on a short story.

            You will be given:
- A short story (including *Precontext*, *Sentence*, and *Ending*)
- A *target word* (homonym)
- A *proposed meaning* for that word

Below are examples that illustrate the scoring rubric for different levels of plausibility, based on similar stories: {combined_text}

Now evaluate the following story and proposed meaning:
**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

**Instructions:**
1.  **Analyze the Context:** Read the complete story and identify all clues that might support or contradict the 'Proposed Meaning'.
2.  **List Evidence For:** State the parts of the story that make the 'Proposed Meaning' plausible.
3.  **List Evidence Against:** State any parts of the story that make the 'Proposed Meaning' implausible.
4.  **Synthesize and Score:** Based on the evidence, provide a final plausibility score using the rubric below.

**Scoring Rubric:**
- **5:** Perfectly plausible. The meaning is strongly and directly supported by multiple clues in the context.
- **4:** Very plausible. The meaning fits well and is consistent with the context.
- **3:** Moderately plausible. The meaning is possible but not strongly implied; the context is ambiguous.
- **2:** Barely plausible. The meaning largely conflicts with the context but isn't entirely impossible.
- **1:** Implausible. The meaning is directly contradicted by the context.


Perform a proper reasoning and only provide the final integer. Any explanation and discussions are not expected..'''                       
        
                    
            # Generate response using the same format as training
            full_text = chat_completion_deepseek(prompt)
            output_updated = full_text.replace('\n', ' ')             
            print(f"Line {line_num}: {output_updated}")
            line_num += 1


if __name__ == "__main__":
    main()

In [ ]:
predicts=[]
with open("Few_shot.txt", "r", encoding="utf-8") as f:
    for i in f:
        elem = i.strip().split(" ")
        if elem[0] == "Line":
            predicts.append(int(elem[2]))
print(predicts)
print(len(predicts))

In [ ]:
import pandas as pd

gold_final_score = []
with open('dev.csv', 'r', encoding='utf-8') as f:
    df = pd.read_csv(f)
    for index, row in df.iterrows():
        average = row['average']
        gold_final_score.append(average)       
        

print(gold_final_score)
print(len(gold_final_score))

In [ ]:
difference = 0
for gold, predict in zip(gold_final_score, predicts):
    diff = abs(gold - predict)
    difference += diff
    
print(difference/len(gold_final_score))

from scipy.stats import spearmanr

# gold_final_score and Gemma_final_score are lists of equal length
spearman_corr, _ = spearmanr(gold_final_score, predicts)
print(f"Spearman Correlation: {spearman_corr:.4f}")


import numpy as np

gold_array = np.array(gold_final_score)
pred_array = np.array(predicts)

std_dev = np.std(gold_array)  # global standard deviation across gold scores
tolerance = std_dev  # "within 1 std deviation"

within_std = np.abs(pred_array - gold_array) <= tolerance
accuracy_within_std = np.mean(within_std)

print(f"Standard Deviation: {std_dev:.4f}")
print(f"Accuracy within 1 Std Dev: {accuracy_within_std:.4f}")

In [ ]:
import csv
def main():
    file_path = 'Updated_dev.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"


            pre_context = f''' **Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Homonym Word:** "{homonym}'''
            
            combined_text, retrieved_texts = rag_pipeline(pre_context)

            prompt=f'''our task is to rate the plausibility of a word's meaning on a scale of 1-5 based on a short story.

            You will be given:
- A short story (including *Precontext*, *Sentence*, and *Ending*)
- A *target word* (homonym)
- A *proposed meaning* for that word

Below are examples that illustrate the scoring rubric for different levels of plausibility, based on similar stories: {combined_text}

Now evaluate the following story and proposed meaning:
**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

**Instructions:**
1.  **Analyze the Context:** Read the complete story and identify all clues that might support or contradict the 'Proposed Meaning'.
2.  **List Evidence For:** State the parts of the story that make the 'Proposed Meaning' plausible.
3.  **List Evidence Against:** State any parts of the story that make the 'Proposed Meaning' implausible.
4.  **Synthesize and Score:** Based on the evidence, provide a final plausibility score using the rubric below.

**Scoring Rubric:**
- **5:** Perfectly plausible. The meaning is strongly and directly supported by multiple clues in the context.
- **4:** Very plausible. The meaning fits well and is consistent with the context.
- **3:** Moderately plausible. The meaning is possible but not strongly implied; the context is ambiguous.
- **2:** Barely plausible. The meaning largely conflicts with the context but isn't entirely impossible.
- **1:** Implausible. The meaning is directly contradicted by the context.


Provide only the final integer score after your reasoning.'''                       
        
                    
            # Generate response using the same format as training
            full_text = get_model_output(prompt)
            output_updated = full_text.replace('\n', ' ')             
            print(f"Line {line_num}: {output_updated}")
            line_num += 1


if __name__ == "__main__":
    main()

In [ ]:
predicts=[]
with open("Few_shot_k2Virtuoso.txt", "r", encoding="utf-8") as f:
    for i in f:
        elem = i.strip().split(" ")
        if elem[0] == "Line":
            predicts.append(int(elem[2]))
print(predicts)
print(len(predicts))
save_predictions_to_jsonl(predicts, "predictions_arcee_k2.jsonl")

In [ ]:
difference = 0
for gold, predict in zip(gold_final_score, predicts):
    diff = abs(gold - predict)
    difference += diff
    
print(difference/len(gold_final_score))

from scipy.stats import spearmanr

# gold_final_score and Gemma_final_score are lists of equal length
spearman_corr, _ = spearmanr(gold_final_score, predicts)
print(f"Spearman Correlation: {spearman_corr:.4f}")


import numpy as np

gold_array = np.array(gold_final_score)
pred_array = np.array(predicts)

std_dev = np.std(gold_array)  # global standard deviation across gold scores
tolerance = std_dev  # "within 1 std deviation"

within_std = np.abs(pred_array - gold_array) <= tolerance
accuracy_within_std = np.mean(within_std)

print(f"Standard Deviation: {std_dev:.4f}")
print(f"Accuracy within 1 Std Dev: {accuracy_within_std:.4f}")

# GPT 40

In [ ]:


import csv
def main():
    file_path = 'Updated_dev.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"

            prompt=f'''our task is to rate the plausibility of a word's meaning on a scale of 1-5 based on a short story.

**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

**Instructions:**
1.  **Analyze the Context:** Read the complete story and identify all clues that might support or contradict the 'Proposed Meaning'.
2.  **List Evidence For:** State the parts of the story that make the 'Proposed Meaning' plausible.
3.  **List Evidence Against:** State any parts of the story that make the 'Proposed Meaning' implausible.
4.  **Synthesize and Score:** Based on the evidence, provide a final plausibility score using the rubric below.

**Scoring Rubric:**
- **5:** Perfectly plausible. The meaning is strongly and directly supported by multiple clues in the context.
- **4:** Very plausible. The meaning fits well and is consistent with the context.
- **3:** Moderately plausible. The meaning is possible but not strongly implied; the context is ambiguous.
- **2:** Barely plausible. The meaning largely conflicts with the context but isn't entirely impossible.
- **1:** Implausible. The meaning is directly contradicted by the context.

Provide only the final integer score after your reasoning.'''                       
        
                    
            # Generate response using the same format as training
            full_text = chat_completion(prompt)
            full_text_fin=full_text.replace('\n', ' ')
            print(f"Line {line_num}: {full_text_fin}")
            line_num += 1


if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd

gold_final_score = []
with open('dev.csv', 'r', encoding='utf-8') as f:
    df = pd.read_csv(f)
    for index, row in df.iterrows():
        average = row['average']
        gold_final_score.append(average)       
        

print(gold_final_score)
print(len(gold_final_score))

In [ ]:
import re

def extract_last_number_from_last_sentence(paragraph):
    # Split paragraph into sentences
    sentences = re.split(r'(?<=[.!?])\s+', paragraph.strip())
    
    if not sentences:
        return None
    
    # Get the last sentence
    last_sentence = sentences[-1]
    
    # Find all numbers (integers or floats)
    numbers = re.findall(r'\d+(?:\.\d+)?', last_sentence)
    
    if numbers:
        # Take the last number found
        num_str = numbers[-1]
        return float(num_str) if '.' in num_str else int(num_str)
    else:
        return None


In [ ]:
predicts=[]
with open("Zeroshot_GPT4o.txt", "r", encoding="utf-8") as f:
    for i in f:
        
        #print(extract_number_from_last_sentence(i))
        predicts.append(extract_last_number_from_last_sentence(i))
        
print(predicts)
print(len(predicts))
#save_predictions_to_jsonl(predicts, "predictions.jsonl")

In [ ]:
#finding incorrect indices
difference = 0
for gold, predict in zip(gold_final_score, predicts):
    diff = abs(gold - predict)
    difference += diff

print(f"Mean Absolute Difference: {difference / len(gold_final_score):.4f}")

from scipy.stats import spearmanr

# Calculate Spearman correlation
spearman_corr, _ = spearmanr(gold_final_score, predicts)
print(f"Spearman Correlation: {spearman_corr:.4f}")

import numpy as np

gold_array = np.array(gold_final_score)
pred_array = np.array(predicts)

# Calculate standard deviation of gold scores
std_dev = np.std(gold_array)
tolerance = std_dev  # within 1 standard deviation

# Determine which predictions are within 1 std deviation
within_std = np.abs(pred_array - gold_array) <= tolerance
accuracy_within_std = np.mean(within_std)

print(f"Standard Deviation: {std_dev:.4f}")
print(f"Accuracy within 1 Std Dev: {accuracy_within_std:.4f}")

# Print all indices where prediction is NOT within tolerance
incorrect_indices = np.where(~within_std)[0]

if len(incorrect_indices) > 0:
    print("\nIndices not within 1 Std Dev:")
    for idx in incorrect_indices:
        print(f"Index {idx}: gold = {gold_array[idx]:.4f}, predict = {pred_array[idx]:.4f}, diff = {abs(gold_array[idx] - pred_array[idx]):.4f}")
else:
    print("\nAll predictions are within 1 Std Dev.")

In [ ]:
difference = 0
for gold, predict in zip(gold_final_score, predicts):
    diff = abs(gold - predict)
    difference += diff
    
print(difference/len(gold_final_score))

from scipy.stats import spearmanr

# gold_final_score and Gemma_final_score are lists of equal length
spearman_corr, _ = spearmanr(gold_final_score, predicts)
print(f"Spearman Correlation: {spearman_corr:.4f}")


import numpy as np

gold_array = np.array(gold_final_score)
pred_array = np.array(predicts)

std_dev = np.std(gold_array)  # global standard deviation across gold scores
tolerance = std_dev  # "within 1 std deviation"

within_std = np.abs(pred_array - gold_array) <= tolerance
accuracy_within_std = np.mean(within_std)

print(f"Standard Deviation: {std_dev:.4f}")
print(f"Accuracy within 1 Std Dev: {accuracy_within_std:.4f}")

- Rag based Evaluation for GPT 40

In [ ]:
import csv
def main():
    file_path = 'Updated_dev.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"


            pre_context = f''' **Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Homonym Word:** "{homonym}'''
            
            combined_text, retrieved_texts = rag_pipeline(pre_context)

            prompt=f'''our task is to rate the plausibility of a word's meaning on a scale of 1-5 based on a short story.

            You will be given:
- A short story (including *Precontext*, *Sentence*, and *Ending*)
- A *target word* (homonym)
- A *proposed meaning* for that word

Below are examples that illustrate the scoring rubric for different levels of plausibility, based on similar stories: {combined_text}

Now evaluate the following story and proposed meaning:
**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

**Instructions:**
1.  **Analyze the Context:** Read the complete story and identify all clues that might support or contradict the 'Proposed Meaning'.
2.  **List Evidence For:** State the parts of the story that make the 'Proposed Meaning' plausible.
3.  **List Evidence Against:** State any parts of the story that make the 'Proposed Meaning' implausible.
4.  **Synthesize and Score:** Based on the evidence, provide a final plausibility score using the rubric below.

**Scoring Rubric:**
- **5:** Perfectly plausible. The meaning is strongly and directly supported by multiple clues in the context.
- **4:** Very plausible. The meaning fits well and is consistent with the context.
- **3:** Moderately plausible. The meaning is possible but not strongly implied; the context is ambiguous.
- **2:** Barely plausible. The meaning largely conflicts with the context but isn't entirely impossible.
- **1:** Implausible. The meaning is directly contradicted by the context.


Provide only the final integer score after your reasoning.'''                       
        
                    
            # Generate response using the same format as training
            full_text = chat_completion(prompt)
            output_updated = full_text.replace('\n', ' ')             
            print(f"Line {line_num}: {output_updated}")
            line_num += 1


if __name__ == "__main__":
    main()

In [ ]:
import csv
def main():
    file_path = 'Updated_dev.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"


            pre_context = f''' **Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Homonym Word:** "{homonym}'''
            
            combined_text, retrieved_texts = rag_pipeline(pre_context)
            #print(combined_text)

            prompt=f'''our task is to rate the plausibility of a word's meaning on a scale of 1-5 based on a short story.

            You will be given:
- A short story (including *Precontext*, *Sentence*, and *Ending*)
- A *target word* (homonym)
- A *proposed meaning* for that word

Below are examples that illustrate the scoring rubric for different levels of plausibility, based on similar stories: {combined_text}

Now evaluate the following story and proposed meaning:
**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

**Instructions:**
1.  **Analyze the Context:** Read the complete story and identify all clues that might support or contradict the 'Proposed Meaning'.
2.  **List Evidence For:** State the parts of the story that make the 'Proposed Meaning' plausible.
3.  **List Evidence Against:** State any parts of the story that make the 'Proposed Meaning' implausible.
4.  **Synthesize and Score:** Based on the evidence, provide a final plausibility score using the rubric below.

**Scoring Rubric:**
- **5:** Perfectly plausible. The meaning is strongly and directly supported by multiple clues in the context.
- **4:** Very plausible. The meaning fits well and is consistent with the context.
- **3:** Moderately plausible. The meaning is possible but not strongly implied; the context is ambiguous.
- **2:** Barely plausible. The meaning largely conflicts with the context but isn't entirely impossible.
- **1:** Implausible. The meaning is directly contradicted by the context.


Provide only the final integer score after your reasoning.'''                       
        
                    
            # Generate response using the same format as training
            full_text = chat_completion(prompt)
            output_updated = full_text.replace('\n', ' ')             
            print(f"Line {line_num}: {output_updated}")
            line_num += 1


if __name__ == "__main__":
    main()

In [ ]:
predicts=[]
with open("Fewshot_k1_gpt.txt", "r", encoding="utf-8") as f:
    for i in f:
        
        #print(extract_last_number_from_last_sentence(i))
        predicts.append(extract_last_number_from_last_sentence(i))
        
print(predicts)
print(len(predicts))

In [ ]:
difference = 0
for gold, predict in zip(gold_final_score, predicts):
    diff = abs(gold - predict)
    difference += diff
    
print(difference/len(gold_final_score))

from scipy.stats import spearmanr

# gold_final_score and Gemma_final_score are lists of equal length
spearman_corr, _ = spearmanr(gold_final_score, predicts)
print(f"Spearman Correlation: {spearman_corr:.4f}")


import numpy as np

gold_array = np.array(gold_final_score)
pred_array = np.array(predicts)

std_dev = np.std(gold_array)  # global standard deviation across gold scores
tolerance = std_dev  # "within 1 std deviation"

within_std = np.abs(pred_array - gold_array) <= tolerance
accuracy_within_std = np.mean(within_std)

print(f"Standard Deviation: {std_dev:.4f}")
print(f"Accuracy within 1 Std Dev: {accuracy_within_std:.4f}")

In [ ]:
predicts=[]
with open("Fewshot_k2_gpt.txt", "r", encoding="utf-8") as f:
    for i in f:
        
        #print(extract_last_number_from_last_sentence(i))
        predicts.append(extract_last_number_from_last_sentence(i))
        
print(predicts)
print(len(predicts))
save_predictions_to_jsonl(predicts, "predictions_gpt_k2.jsonl")

In [ ]:
difference = 0
for gold, predict in zip(gold_final_score, predicts):
    diff = abs(gold - predict)
    difference += diff
    
print(difference/len(gold_final_score))

from scipy.stats import spearmanr

# gold_final_score and Gemma_final_score are lists of equal length
spearman_corr, _ = spearmanr(gold_final_score, predicts)
print(f"Spearman Correlation: {spearman_corr:.4f}")


import numpy as np

gold_array = np.array(gold_final_score)
pred_array = np.array(predicts)

std_dev = np.std(gold_array)  # global standard deviation across gold scores
tolerance = std_dev  # "within 1 std deviation"

within_std = np.abs(pred_array - gold_array) <= tolerance
accuracy_within_std = np.mean(within_std)

print(f"Standard Deviation: {std_dev:.4f}")
print(f"Accuracy within 1 Std Dev: {accuracy_within_std:.4f}")

In [ ]:
#Gemini
import time
import csv
def main():
    file_path = 'Updated_dev.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"


            pre_context = f''' **Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Homonym Word:** "{homonym}'''
            
            combined_text, retrieved_texts = rag_pipeline(pre_context)

            prompt=f'''Your task is to rate the plausibility of a word's meaning on a scale of 1-5 based on a short story.

            You will be given:
- A short story (including *Precontext*, *Sentence*, and *Ending*)
- A *target word* (homonym)
- A *proposed meaning* for that word

Below are examples that illustrate the scoring rubric for different levels of plausibility, based on similar stories: {combined_text}

Now evaluate the following story and proposed meaning:
**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

**Instructions:**
1.  **Analyze the Context:** Read the complete story and identify all clues that might support or contradict the 'Proposed Meaning'.
2.  **List Evidence For:** State the parts of the story that make the 'Proposed Meaning' plausible.
3.  **List Evidence Against:** State any parts of the story that make the 'Proposed Meaning' implausible.
4.  **Synthesize and Score:** Based on the evidence, provide a final plausibility score using the rubric below.

**Scoring Rubric:**
- **5:** Perfectly plausible. The meaning is strongly and directly supported by multiple clues in the context.
- **4:** Very plausible. The meaning fits well and is consistent with the context.
- **3:** Moderately plausible. The meaning is possible but not strongly implied; the context is ambiguous.
- **2:** Barely plausible. The meaning largely conflicts with the context but isn't entirely impossible.
- **1:** Implausible. The meaning is directly contradicted by the context.


Provide only the final integer score after your reasoning.'''                       
        
                    
            # Generate response using the same format as training
            full_text = get_model_output_gemini(prompt)
            output_updated = full_text.replace('\n', ' ')             
            print(f"Line {line_num}: {output_updated}")
            line_num += 1
            time.sleep(10)


if __name__ == "__main__":
    main()

In [ ]:
predicts=[]
with open("FewShot_k2_Gemini.txt", "r", encoding="utf-8") as f:
    for i in f:
        no = get_number_at_end(i.strip())
        predicts.append(no)
        #print(no)
print(predicts)
print(len(predicts))

In [ ]:
difference = 0
for gold, predict in zip(gold_final_score, predicts):
    diff = abs(gold - predict)
    difference += diff
    
print(difference/len(gold_final_score))

from scipy.stats import spearmanr

# gold_final_score and Gemma_final_score are lists of equal length
spearman_corr, _ = spearmanr(gold_final_score, predicts)
print(f"Spearman Correlation: {spearman_corr:.4f}")


import numpy as np

gold_array = np.array(gold_final_score)
pred_array = np.array(predicts)

std_dev = np.std(gold_array)  # global standard deviation across gold scores
tolerance = std_dev  # "within 1 std deviation"

within_std = np.abs(pred_array - gold_array) <= tolerance
accuracy_within_std = np.mean(within_std)

print(f"Standard Deviation: {std_dev:.4f}")
print(f"Accuracy within 1 Std Dev: {accuracy_within_std:.4f}")

In [ ]:
import pandas as pd
predicts=[]
gold_final_score = []
with open('dev.csv', 'r', encoding='utf-8') as f:
    df = pd.read_csv(f)
    for index, row in df.iterrows():
        average = row['average']
        point = round(average)
        predicts.append(point)
        gold_final_score.append(average)       
        

print(gold_final_score)
print(len(gold_final_score))

In [ ]:
difference = 0
for gold, predict in zip(gold_final_score, predicts):
    diff = abs(gold - predict)
    difference += diff
    
print(difference/len(gold_final_score))

from scipy.stats import spearmanr

# gold_final_score and Gemma_final_score are lists of equal length
spearman_corr, _ = spearmanr(gold_final_score, predicts)
print(f"Spearman Correlation: {spearman_corr:.4f}")


import numpy as np

gold_array = np.array(gold_final_score)
pred_array = np.array(predicts)

std_dev = np.std(gold_array)  # global standard deviation across gold scores
tolerance = std_dev  # "within 1 std deviation"

within_std = np.abs(pred_array - gold_array) <= tolerance
accuracy_within_std = np.mean(within_std)

print(f"Standard Deviation: {std_dev:.4f}")
print(f"Accuracy within 1 Std Dev: {accuracy_within_std:.4f}")

In [ ]:
# O4 Mini : k=5

In [ ]:
import csv
def main():
    file_path = 'Updated_dev.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"


            pre_context = f''' **Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Homonym Word:** "{homonym}'''
            
            combined_text, retrieved_texts = rag_pipeline(pre_context)

            prompt=f'''Your task is to rate the plausibility of a word's meaning on a scale of 1-5 based on a short story. Remember you will simulate the human reasoning and rating process by carefully considering the context provided.

            You will be given:
- A short story (including *Precontext*, *Sentence*, and *Ending*)
- A *target word* (homonym)
- A *proposed meaning* for that word

Below are examples that illustrate the scoring rubric for different levels of plausibility, based on similar stories: {combined_text}

Now evaluate the following story and proposed meaning:
**Story:**
- Precontext: {precontext}
- Sentence: {sentence}
- Ending: {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

**Instructions:**
1.  **Analyze the Context:** Read the complete story and identify all clues that might support or contradict the 'Proposed Meaning'.
2.  **List Evidence For:** State the parts of the story that make the 'Proposed Meaning' plausible.
3.  **List Evidence Against:** State any parts of the story that make the 'Proposed Meaning' implausible.
4.  **Synthesize and Score:** Based on the evidence, provide a final plausibility score using the rubric below.

**Scoring Rubric:**
- **5:** Perfectly plausible. The meaning is strongly and directly supported by multiple clues in the context.
- **4:** Very plausible. The meaning fits well and is consistent with the context.
- **3:** Moderately plausible. The meaning is possible but not strongly implied; the context is ambiguous.
- **2:** Barely plausible. The meaning largely conflicts with the context but isn't entirely impossible.
- **1:** Implausible. The meaning is directly contradicted by the context.


Provide only the final integer score after your reasoning.'''                       
        
                    
            # Generate response using the same format as training
            full_text = chat_completion_o4(prompt)
            output_updated = full_text.replace('\n', ' ')             
            print(f"Line {line_num}: {output_updated}")
            line_num += 1


if __name__ == "__main__":
    main()

In [ ]:
import re

def get_number_at_end(sentence: str):
    """
    Returns the number found at the end of a given sentence.
    Handles cases like **5**, (5), or 5.
    Returns None if no number is found.
    """
    match = re.search(r'[\*\(\[\{]*(-?\d+(?:\.\d+)?)[\*\)\]\}]*\s*$', sentence)
    if match:
        num_str = match.group(1)
        # Return int if it’s whole, float otherwise
        return int(num_str) if num_str.isdigit() or (num_str.startswith('-') and num_str[1:].isdigit()) else float(num_str)
    return None

In [ ]:
predicts=[]
with open("Few_shot_k5_gpt.txt", "r", encoding="utf-8") as f:
    for i in f:
        no = get_number_at_end(i.strip())
        predicts.append(no)
        #print(no)
print(predicts)
print(len(predicts))
save_predictions_to_jsonl(predicts, "predictions.jsonl")

In [ ]:
difference = 0
for gold, predict in zip(gold_final_score, predicts):
    diff = abs(gold - predict)
    difference += diff
    
print(difference/len(gold_final_score))

from scipy.stats import spearmanr

# gold_final_score and Gemma_final_score are lists of equal length
spearman_corr, _ = spearmanr(gold_final_score, predicts)
print(f"Spearman Correlation: {spearman_corr:.4f}")


import numpy as np

gold_array = np.array(gold_final_score)
pred_array = np.array(predicts)

std_dev = np.std(gold_array)  # global standard deviation across gold scores
tolerance = std_dev  # "within 1 std deviation"

within_std = np.abs(pred_array - gold_array) <= tolerance
accuracy_within_std = np.mean(within_std)

print(f"Standard Deviation: {std_dev:.4f}")
print(f"Accuracy within 1 Std Dev: {accuracy_within_std:.4f}")

In [ ]:
import csv
def main():
    file_path = 'Updated_dev.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"

            prompt=f'''You are asked to rate how plausible a proposed meaning of a target word (a homonym) is in a story. Use careful, structured reasoning but only output a single integer (1, 2, 3, 4, or 5) — nothing else.
                    You will be given: 
                     - A short story (including *Precontext*, *Sentence*, and *Ending* :may be NO ENDING if not present)
                     - A *target word* (homonym)
                     - A *proposed meaning* for that word                    

Task: Decide how plausible the Judged Meaning of the Homonym is in the story context, and output only one integer (1–5) as the final plausibility score.
Required procedure are provided after the story.(do all steps mentally / privately — do not print them; print only the final integer):

Input fields:
                    Precontext: {precontext}
                    Sentence (contains homonym): {sentence}
                    Ending: {ending} 
                    Homonym: {homonym}
                    Judged Meaning: {judged_meaning}


1. Verify context completeness & note missing ending: If ending is NO ENDING (missing), treat contextual evidence as weaker; avoid overconfident extremes unless the sentence+precontext strongly selects one sense. Missing ending → introduce slight uncertainty bias (prefer 3 over 4/2 unless strong cues exist).
2. Identify literal vs figurative / idiomatic cues: 
- Look for physical objects, locations, bodily references, motion verbs, measurement words → favors literal senses.
- Look for evaluative/adjectival phrases, emotions, idiomatic constructions, metaphors, or social roles → favors figurative/abstract senses.
3. Search for collocational and syntactic signals:
- Nearby nouns, verbs, prepositions or objects that commonly select one sense (e.g., “rail,” “sleeper,” “railway” → rail sense of track; “dribbling” + “soccer ball” → sports sense).
- If the sentence itself strongly selects a sense (subject/object congruence), weight it heavily.
4. Weigh ending vs precontext vs sentence:
-Rank evidence strength: Sentence (strongest) > Precontext > Ending (if present).
-However, a specific ending that explicitly implies a sense should override ambiguous sentence cues.
5. Corner-case rules (explicit tie-breakers and calibration):
- If both senses are clearly plausible (balanced evidence) → score 3 (moderate plausibility).
- If evidence slightly favors the judged meaning but not strongly → score 4.
- If evidence slightly disfavors it but does not contradict → score 2.
- If judged meaning is directly contradicted by sentence or precontext → score 1.
- If there is strong, multiple-clue support (sentence + precontext and/or explicit ending) → score 5.
- If ending == NO ENDING and cues are mixed → bias towards 3 (avoid 4/2 unless strong single-source evidence).
6. If story parts conflict (e.g., precontext implies one sense, ending another): Use 3 unless one part is very explicit — then choose 2 or 4.
7. If unsure or context is weak: Default to 3 rather than guessing.

Scoring rubric reminder (for internal use only):
5 = Perfectly plausible; strong direct support.
4 = Very plausible; fits well, consistent.
3 = Moderately plausible / ambiguous.
2 = Barely plausible; largely conflicts but not impossible.
1 = Implausible; directly contradicted. 

Final output rule:
After reasoning, output exactly one integer: 1, 2, 3, 4, or 5. No text, no punctuation, no explanation.'''                       
        
                    
            # Generate response using the same format as training
            full_text = chat_completion_4o(prompt)
            full_text_fin=full_text.replace('\n', ' ')
            print(f"Line {line_num}: {full_text_fin}")
            line_num += 1


if __name__ == "__main__":
    main()

In [ ]:
predicts=[]
with open("Zeroshot_gpt_updated.txt", "r", encoding="utf-8") as f:
    for i in f:
        no = i.split(":")[1].strip()
        no= int(no)
        predicts.append(no)
        #print(no)
print(predicts)
print(len(predicts))

In [ ]:
difference = 0
for gold, predict in zip(gold_final_score, predicts):
    diff = abs(gold - predict)
    difference += diff
    
print(difference/len(gold_final_score))

from scipy.stats import spearmanr

# gold_final_score and Gemma_final_score are lists of equal length
spearman_corr, _ = spearmanr(gold_final_score, predicts)
print(f"Spearman Correlation: {spearman_corr:.4f}")


import numpy as np

gold_array = np.array(gold_final_score)
pred_array = np.array(predicts)

std_dev = np.std(gold_array)  # global standard deviation across gold scores
tolerance = std_dev  # "within 1 std deviation"

within_std = np.abs(pred_array - gold_array) <= tolerance
accuracy_within_std = np.mean(within_std)

print(f"Standard Deviation: {std_dev:.4f}")
print(f"Accuracy within 1 Std Dev: {accuracy_within_std:.4f}")

In [ ]:
import csv
def main():
    file_path = 'Updated_dev.csv'
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"
            
            pre_context = f''' **Story:**  Precontext: {precontext} - Sentence: {sentence} - Ending: {ending}  Homonym Word: "{homonym}'''
            combined_text, retrieved_texts = rag_pipeline(pre_context)

            prompt=f'''Your task is to rate the plausibility of a word's meaning on a scale of 1-5 
based on a short story. You must follow the Thinking Process below to arrive at your score.

You will be provided with few example stories that illustrate the scoring rubric for different levels of plausibility, based on similar stories: {combined_text}.

Now evaluate the following story and proposed meaning:

**Story:**
* **Precontext:** {precontext}
* **Sentence:** {sentence}
* **Ending:** {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

### Thinking Process & Response Format

*Complete each step of this process in your analysis.*

**Step 1: Initial Context Analysis**
* Read the **Precontext** *alone*. Which meaning(s) of "{homonym}" does it make you expect?
* Read the **Ambiguous Sentence** *alone*. Is the homonym part of a common idiom (e.g., "had guts") or a set phrase (e.g., "drawing water")?

**Step 2: The Ending as a Clarifier**
* Now, read the **Ending**. How does this new information *shift* the story?
* Does the Ending strongly *confirm* one meaning, *contradict* another, or introduce a *twist*?
* **Crucially:** Does the Ending act as a *paraphrase* or *definition* of a figurative or idiomatic meaning (e.g., an ending of "She was very brave" is a direct definition for the meaning "fortitude")?

**Step 3: Holistic Coherence Check**
* Read all three parts (Precontext, Sentence, Ending) as a *single, coherent narrative*.
* Can all parts co-exist with the **Proposed Meaning**? Or do some parts create a conflict?
* **Supporting Clues:** List the specific words/phrases from the *entire* story (Precontext, Sentence, or Ending) that support the **Proposed Meaning**.
* **Conflicting Clues:** List the specific words/phrases that contradict or make the **Proposed Meaning** awkward or implausible.

**Step 4: Final Score and Justification**
* Based on your coherence check in Step 3, assign a score.
* *Pay close attention to contradictions*: If a strong clue in the Ending (e.g., "wetness on his chin") directly conflicts with a meaning ("propel a ball"), the score must be low, *even if* the precontext ("soccer ball") supported it.
* *Pay close attention to modifiers*: If the sentence just describes *how* an action is done (e.g., "pick *without discernment*"), the core action ("select") is still plausible.

**Scoring Rubric:**
* **5: Perfectly plausible.** The meaning is strongly supported by the *entire* context, and all parts of the story (Precontext, Sentence, Ending) form a consistent, logical narrative.
* **4: Very plausible.** The meaning fits well and is consistent. There might be minor ambiguity, but no real contradictions.
* **3: Moderately plausible.** The meaning is possible, but the context is ambiguous or contains *minor* conflicting clues that make the story a bit strange.
* **2: Barely plausible.** The meaning *largely* conflicts with the context. You have to ignore a key piece of information (especially in the Ending) to make it fit.
* **1: Implausible.** The meaning is directly and strongly contradicted by the context (e.g., the Ending makes it physically impossible or nonsensical).

Do all the reasoning *mentally* / *privately* — do not print it; print only the final integer score as a output.
'''
        
                    
            # Generate response using the same format as training
            full_text = chat_completion_4o(prompt)
            full_text_fin=full_text.replace('\n', ' ')
            print(f"Line {line_num}: {full_text_fin}")
            line_num += 1


if __name__ == "__main__":
    main()

In [ ]:
predicts=[]
with open("fewshot_updated_gpt.txt", "r", encoding="utf-8") as f:
    for i in f:
        no = i.split(":")[1].strip()
        no= int(no)
        predicts.append(no)
        #print(no)
print(predicts)
print(len(predicts))

In [ ]:
difference = 0
for gold, predict in zip(gold_final_score, predicts):
    diff = abs(gold - predict)
    difference += diff
    
print(difference/len(gold_final_score))

from scipy.stats import spearmanr

# gold_final_score and Gemma_final_score are lists of equal length
spearman_corr, _ = spearmanr(gold_final_score, predicts)
print(f"Spearman Correlation: {spearman_corr:.4f}")


import numpy as np

gold_array = np.array(gold_final_score)
pred_array = np.array(predicts)

std_dev = np.std(gold_array)  # global standard deviation across gold scores
tolerance = std_dev  # "within 1 std deviation"

within_std = np.abs(pred_array - gold_array) <= tolerance
accuracy_within_std = np.mean(within_std)

print(f"Standard Deviation: {std_dev:.4f}")
print(f"Accuracy within 1 Std Dev: {accuracy_within_std:.4f}")

In [ ]:
#print the distribution of predicted scores

In [ ]:
import pandas as pd
gold_predicts=[]
gold_final_score = []
with open('dev.csv', 'r', encoding='utf-8') as f:
    df = pd.read_csv(f)
    for index, row in df.iterrows():
        average = row['average']
        point = round(average)
        gold_predicts.append(point)
        gold_final_score.append(average)       
        

print(gold_final_score)
print(len(gold_final_score))
print(gold_predicts)

In [ ]:
one=0
two=0
three=0
four=0
five=0
for i in gold_predicts:
    if i==1:
        one+=1
    elif i==2:
        two+=1
    elif i==3:
        three+=1
    elif i==4:
        four+=1
    elif i==5:
        five+=1
print(f"1: {one}, 2: {two}, 3: {three}, 4: {four}, 5: {five}")
    

In [ ]:
one=0
two=0
three=0
four=0
five=0
for i,j in zip(gold_predicts,gptpredicts):
    if i==1 and j==1:
        one+=1
    elif i==2 and j==2:
        two+=1
    elif i==3 and j==3:
        three+=1
    elif i==4 and j==4:
        four+=1
    elif i==5 and j==5:
        five+=1
print(f"1: {one}, 2: {two}, 3: {three}, 4: {four}, 5: {five}")

In [ ]:
virpredicts=[]
with open("Zeroshot_Virtuoso.txt", "r", encoding="utf-8") as f:
    for i in f:
        elem = i.strip().split(" ")
        if elem[0] == "Line":
            virpredicts.append(int(elem[2]))
print(virpredicts)
print(len(virpredicts))

In [ ]:
gempredicts=[]
with open("zeroshot_Gemini.txt", "r", encoding="utf-8") as f:
    for i in f:
        no = get_number_at_end(i.strip())
        gempredicts.append(no)
        #(no)
print(gempredicts)
print(len(gempredicts))
#save_predictions_to_jsonl(gempredicts, "predictions.jsonl")

In [ ]:
gptpredicts=[]
with open("Zeroshot_GPT4o.txt", "r", encoding="utf-8") as f:
    for i in f:
        
        #print(extract_number_from_last_sentence(i))
        gptpredicts.append(extract_last_number_from_last_sentence(i))

print(gptpredicts)
print(len(gptpredicts))

In [ ]:
import matplotlib.pyplot as plt

def plot_predictions_distribution(gold_predicts, virpredicts, gempredicts, gptpredicts):
    """
    Plots a line graph comparing the distributions of four prediction result lists:
    - gold_predicts: Ground truth values
    - virpredicts: Virtuosa Large model predictions
    - gempredicts: Gemini model predictions
    - gptpredicts: GPT-40 model predictions
    """
    
    plt.figure(figsize=(10, 6))
    
    # Plot each line
    plt.plot(gold_predicts, label='Golden Results', marker='o')
    plt.plot(virpredicts, label='Virtuosa Large', marker='s')
    plt.plot(gempredicts, label='Gemini', marker='^')
    plt.plot(gptpredicts, label='GPT-40', marker='d')
    
    # Add labels, title, legend, and grid
    plt.title('Model Predictions Distribution Comparison')
    plt.xlabel('Sample Index')
    plt.ylabel('Prediction Value')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    
    # Improve layout and show the plot
    plt.tight_layout()
    plt.show()

In [ ]:
plot_predictions_distribution(gold_predicts, virpredicts, gempredicts, gptpredicts)

In [ ]:
import matplotlib.pyplot as plt

def plot_predictions_hist(gold_predicts, virpredicts, gempredicts, gptpredicts):
    """
    Plot histogram distributions for four prediction lists.
    """
    plt.figure(figsize=(8, 6))

    plt.hist(gold_predicts, bins=range(1, 7), alpha=0.5, label='Golden Results', edgecolor='black')
    plt.hist(virpredicts, bins=range(1, 7), alpha=0.5, label='Virtuosa Large', edgecolor='black')
    plt.hist(gempredicts, bins=range(1, 7), alpha=0.5, label='Gemini', edgecolor='black')
    plt.hist(gptpredicts, bins=range(1, 7), alpha=0.5, label='GPT-40', edgecolor='black')

    plt.title('Distribution of Prediction Scores')
    plt.xlabel('Prediction Value (1–5)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


In [ ]:
plot_predictions_hist(gold_predicts, virpredicts, gempredicts, gptpredicts)

In [ ]:
predicts=[]
for i,j,k in zip(virpredicts, gempredicts, gptpredicts):
    avg = (i + j + k) / 3
    final_point = round(avg)
    predicts.append(final_point)
print(predicts)

In [ ]:
predicts=[]
for i,j,k in zip(virpredicts, gempredicts, gptpredicts):
    list_vals = [i, j, k]
    min_val = min(list_vals)   
    predicts.append(min_val)
print(predicts)

In [ ]:
predicts=[]
for i,j,k in zip(virpredicts, gempredicts, gptpredicts):
    list_vals = [i, j, k]
    max_val = max(list_vals)
    predicts.append(max_val)
print(predicts)

In [ ]:
difference = 0
for gold, predict in zip(gold_final_score, predicts):
    diff = abs(gold - predict)
    difference += diff
    
print(difference/len(gold_final_score))

from scipy.stats import spearmanr

# gold_final_score and Gemma_final_score are lists of equal length
spearman_corr, _ = spearmanr(gold_final_score, predicts)
print(f"Spearman Correlation: {spearman_corr:.4f}")


import numpy as np

gold_array = np.array(gold_final_score)
pred_array = np.array(predicts)

std_dev = np.std(gold_array)  # global standard deviation across gold scores
tolerance = std_dev  # "within 1 std deviation"

within_std = np.abs(pred_array - gold_array) <= tolerance
accuracy_within_std = np.mean(within_std)

print(f"Standard Deviation: {std_dev:.4f}")
print(f"Accuracy within 1 Std Dev: {accuracy_within_std:.4f}")